In [ ]:
import os
import struct
import gc
import numpy as np
from keras.utils import np_utils
import keras
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

# 1 . Create Dataset -- add random noise pics into MNIST

In [ ]:
from keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data() 

X_train = X_train / 255
X_test  = X_test / 255

In [ ]:
def gen_add_noise_50(X_train,y_train):
    noise_random = np.random.rand(50000,32,32,3)
    noise_labels = np.full((50000,1), 100, dtype=int)
    X_train_50 = np.concatenate((X_train, noise_random), axis = 0 )
    y_train_50 = np.concatenate((y_train, noise_labels), axis = 0 )
    
    return X_train_50,y_train_50

In [ ]:
def gen_add_noise_5(X_train,y_train):
    noise_random_5 = np.random.rand(500,32,32,3)
    noise_labels_5 = np.full((500,1), 100, dtype=int)
    
    X_train_5 = np.concatenate((X_train, noise_random_5), axis = 0 )
    y_train_5 = np.concatenate((y_train, noise_labels_5), axis = 0 )
    
    return X_train_5,y_train_5

In [ ]:
def shuffle_me(X_train,y_train):
    ind_list = [i for i in range(len(X_train))]
    np.random.shuffle(ind_list)
    X_train  = X_train[ind_list, :,:,:]
    y_train = y_train[ind_list,]
    
    return X_train,y_train

In [ ]:
def keras_style_ds(X_train, X_test, y_train, y_test,original):
# keras - input

    if(original == 1):
        num_classes = 100               
    else:
        num_classes = 101
        
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    
    return X_train, X_test, y_train, y_test, num_classes

# 2 . Build Model -- keras model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta



In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
class TestCallback(keras.callbacks.Callback):
    
    def __init__(self, test_data,accs,losses):
        self.test_data = test_data
        self.accs = accs
        self.losses = losses

    def on_epoch_end(self, epoch, logs={}):

        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        accs.append(acc)
        losses.append(loss)

        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


In [ ]:
class EarlyStoppingByLossVal(keras.callbacks.Callback):
    def __init__(self, monitor='val_loss', patience=10, mode='auto', min_delta=0.01, verbose=0):
        super(keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.patience = patience
        self.mode = mode
        self.min_delta = min_delta
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            print("Early stopping requires %s available!" % self.monitor)
            exit()

        if current < self.min_delta:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
def model_factory():
    K.clear_session()

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     data_format='channels_last',
                     input_shape=(32, 32, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
# model = model_factory()
# model.summary()

# 3. Training

In [ ]:
# Train no random noise added
hs_mnist_no_scores = []
hs_mnist_no_history= []
for i in range(1):
    X_train_no,y_train_no = X_train,y_train
    X_train_no,y_train_no = shuffle_me(X_train_no,y_train_no)
    X_train_no, X_test_no, y_train_no, y_test_no, num_classes = keras_style_ds(X_train_no, X_test, 
                                                                               y_train_no, y_test, 1)

    history_loss_no = LossHistory()
    early_stopping_no = EarlyStoppingByLossVal()
    accs = []
    losses = []
    test_acc_loss_no = TestCallback([X_test_no, y_test_no],accs,losses)

    model = model_factory()
#     model.summary()
    
    history_no = model.fit(X_train_no, y_train_no, 
                           epochs=100, batch_size=32,
                           validation_split=0.1,verbose=1,
                           callbacks=[history_loss_no,early_stopping_no,test_acc_loss_no])
    hs_mnist_no_history.append(history_no)
    
    model.save("./models/CIFAR_dnn_all_convergen_validation10_test_0425_no_%d.h5" % (i+1))
    
    print()
    print(test_acc_loss_no.accs)
    print(test_acc_loss_no.losses)
    
    print()
    gc.collect()

In [ ]:
# Train with 60,000 random noise added
hs_mnist_50_scores = []
hs_mnist_50_history= []
for h in range(1):
    
    X_train_50,y_train_50 = gen_add_noise_50(X_train,y_train)
    X_train_50,y_train_50 = shuffle_me(X_train_50,y_train_50)
    X_train_50, X_test_50, y_train_50, y_test_50, num_classes = keras_style_ds(X_train_50, X_test, 
                                                                               y_train_50, y_test, 0)
    
    accs = []
    losses = []
    
    history_loss_50 = LossHistory()
    early_stopping_50 = EarlyStoppingByLossVal() 
    test_acc_loss_50 = TestCallback([X_test_50, y_test_50], accs, losses)
    
    model_50 = model_factory()
#     model_60.summary()
    
    history_50 = model_50.fit(X_train_50, y_train_50, 
                              epochs=100, batch_size=32,
                              validation_split=0.1,verbose=1,
                             callbacks=[history_loss_50,early_stopping_50,test_acc_loss_50])
    hs_mnist_50_history.append(history_50)
    
    model_50.save("./models/CIFAR_dnn_all_convergen_validation10_test_0425_50_test_%d.h5" % (h+1))

    print()
    print(test_acc_loss_50.accs)
    print(test_acc_loss_50.losses)
    print()
   
    gc.collect()

In [ ]:
# Train with 50,000 random noise added
hs_mnist_5_history= []
for h in range(1):
    
    X_train_5,y_train_5 = gen_add_noise_5(X_train,y_train)
    X_train_5,y_train_5 = shuffle_me(X_train_5,y_train_5)
    X_train_5, X_test_5, y_train_5, y_test_5, num_classes = keras_style_ds(X_train_5, X_test, 
                                                                           y_train_5, y_test, 0)
    
    accs = []
    losses = []
    
    history_loss_5 = LossHistory()
    early_stopping_5 = EarlyStoppingByLossVal() 
    test_acc_loss_5 = TestCallback([X_test_5, y_test_5], accs, losses)
    
    model_5 = model_factory()
#     model_60.summary()
    
    history_5 = model_5.fit(X_train_5, y_train_5, 
                              epochs=100, batch_size=32,
                              validation_split=0.1,verbose=1,
                             callbacks=[history_loss_5,early_stopping_5,test_acc_loss_5])
    hs_mnist_5_history.append(history_5)
    
    model_5.save("./models/CIFAR_dnn_all_convergen_validation10_test_0425_5_test_%d.h5" % (h+1))

    print()
    print(test_acc_loss_5.accs)
    print(test_acc_loss_5.losses)
    print()
   
    gc.collect()